In [1]:
import os
os.chdir("../")

In [10]:
from dataclasses import dataclass
from pathlib import Path
import pandas as pd
import os,sys
from src.logger.custom_logging import logger
from sklearn.ensemble import RandomForestClassifier
import joblib
from src.exceptions.expection import CustomException
from sklearn.metrics import accuracy_score



@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_model_path: Path
    training_data_path: Path
    testing_data_path: Path
    

In [11]:

from src.utils.utlis import read_yaml,create_directories
from src.constants import *

In [12]:
class ConfigManager:
    def __init__(self,
                  config_filepath = CONFIG_FILE_PATH,
                params_filepath = PARAMS_FILE_PATH
  ):
        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)


        create_directories([self.config.artifacts_root])


    def get_model_trainer_config(self)-> ModelTrainerConfig:
        config=self.config.model_trainer
        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            train_model_path=config.train_model_path,
            training_data_path=config.training_data_path,
            testing_data_path=config.testing_data_path
    

        )    
        return model_trainer_config

In [13]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    @staticmethod
    def save_model(path: Path, model):
        """Save the trained model to the specified path."""
        try:
            joblib.dump(model, path)
            print(f"Model saved at {path}")
        except Exception as e:
            print(f"Error saving model: {e}")
            logger.error(f"Error saving model: {e}")
            raise CustomException(e, sys)      


    
    def train(self):
        train_data=pd.read_csv(self.config.training_data_path)
        test_data=pd.read_csv(self.config.testing_data_path)
        try:
            X_train = train_data.iloc[:, :-1].values
            y_train = train_data.iloc[:, -1].values
            X_test = test_data.iloc[:, :-1].values
            y_test = test_data.iloc[:, -1].values   

            model = RandomForestClassifier()
            
            model.fit(X_train, y_train)
            y_pred = model.predict(X_test)
            
            test_model_score = accuracy_score(y_test, y_pred)

            print(f"Test model score: {test_model_score}")

            # Save the best model
            self.save_model(path=self.config.train_model_path,model=model)
        except Exception as e:
            logger.error(f'Error occurred: {e}')
            raise CustomException(e, sys)

In [14]:
try:
    config=ConfigManager()
    model_trainer_config=config.get_model_trainer_config()
    model_trainer=ModelTrainer(model_trainer_config)
    model_trainer.train()

except Exception as e:
    raise CustomException(e,sys)

Test model score: 1.0
Model saved at artifacts/model_trainer/model.pkl
